In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
alcohol_csv = "Resources/Alcohol_sales.csv"
alcohol_data = pd.read_csv(alcohol_csv)
alcohol_data.head()

,DATE,S4248SM144NCEN
0,1/1/1992,3459
1,2/1/1992,3458
2,3/1/1992,4002
3,4/1/1992,4564
4,5/1/1992,4221


In [3]:
alcohol_clean = alcohol_data.rename(columns={"DATE": "monthrecorded", "S4248SM144NCEN": "monthlysales"})
alcohol_clean.head()

,monthrecorded,monthlysales
0,1/1/1992,3459
1,2/1/1992,3458
2,3/1/1992,4002
3,4/1/1992,4564
4,5/1/1992,4221


In [4]:
weather_csv = "Resources/Weather_Data.csv"
weather_data = pd.read_csv(weather_csv)
weather_data.head()

,Contiguous U.S. Average Temperature,Unnamed: 1,Unnamed: 2
0,Units: Degrees Fahrenheit,NaN,NaN
1,Base Period: 1901-2000,NaN,NaN
2,Missing: -99,NaN,NaN
3,Date,Value,Anomaly
4,200001,33.57,3.45


In [5]:
weather = weather_data[["Contiguous U.S.  Average Temperature", "Unnamed: 1"]].copy()
weather.head()

,Contiguous U.S. Average Temperature,Unnamed: 1
0,Units: Degrees Fahrenheit,NaN
1,Base Period: 1901-2000,NaN
2,Missing: -99,NaN
3,Date,Value
4,200001,33.57


In [6]:
weather_clean = weather.drop([0,1,2,3], axis=0)
weather_clean.head()

,Contiguous U.S. Average Temperature,Unnamed: 1
4,200001,33.57
5,200002,39.78
6,200003,46.04
7,200004,52.59
8,200005,63.14


In [7]:
weather_table = weather_clean.rename(columns={"Contiguous U.S.  Average Temperature": "monthtemp", "Unnamed: 1": "avgtemp"})
weather_table.head()

,monthtemp,avgtemp
4,200001,33.57
5,200002,39.78
6,200003,46.04
7,200004,52.59
8,200005,63.14


In [8]:
rds_connection_string = "postgres:Ranger@localhost:5432/Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [9]:
engine.table_names()

['temperature', 'drinking', 'temps_alcoholsales']

In [10]:
weather_table.to_sql(name='temperature', con=engine, if_exists='append', index=False)

In [11]:
alcohol_clean.to_sql(name='drinking', con=engine, if_exists='append', index=False)

In [12]:
pd.read_sql_query('select * from temperature', con=engine).head()

,id,monthtemp,avgtemp
0,241,200001,33.57
1,242,200002,39.78
2,243,200003,46.04
3,244,200004,52.59
4,245,200005,63.14


In [13]:
pd.read_sql_query('select * from drinking', con=engine).head()

,id,monthrecorded,monthlysales
0,1,1992-01-01,3459
1,2,1992-02-01,3458
2,3,1992-03-01,4002
3,4,1992-04-01,4564
4,5,1992-05-01,4221


In [22]:
df = pd.read_sql_query('select * from temps_alcoholsales', con=engine)

In [23]:
temps_sales = df.drop(['id', 'monthrecorded'], axis=1)
temps_sales.head()

,monthtemp,avgtemp,monthlysales
0,2000-01-01,33.57,4243
1,2000-02-01,39.78,4952
2,2000-03-01,46.04,6008
3,2000-04-01,52.59,5353
4,2000-05-01,63.14,6435


In [24]:
temp_sales_clean = temps_sales.rename(columns={"monthtemp": "Month", "avgtemp": "Avg Temp (F)",
                                               "monthlysales": "Alcohol Sales (in millions)"})
temp_sales_clean

,Month,Avg Temp (F),Alcohol Sales (in millions)
0,2000-01-01,33.57,4243
1,2000-02-01,39.78,4952
2,2000-03-01,46.04,6008
3,2000-04-01,52.59,5353
4,2000-05-01,63.14,6435
...,...,...,...
234,2019-07-01,74.61,13724
235,2019-08-01,73.74,14464
236,2019-09-01,68.43,13215
237,2019-10-01,52.05,14680
